In [5]:
import os

import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
import torch.optim as optim


from datetime import datetime
from torch import nn
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import MessagePassing, DataParallel
from torch_scatter import scatter
from torch.utils.tensorboard import SummaryWriter

from DataClasses import lmdb_dataset, Dataset, DataListLoader
from ModelFunctions import train, evaluate, inference

In [6]:
import sys
sys.path.append(os.path.expanduser('../ocpmodels/models'))
sys.path.append(os.path.expanduser('../../ocp_airi'))

from spinconv_with_solid_angles import spinconv
#from spinconv_with_radii import spinconv
#from spinconv import spinconv

In [7]:
#вызывается каждый раз, когда датасет отдаёт элемент (систему)
#делаем из данных матрицу векторов-атомов, список рёбер (edge_index) и матрицу векторов-рёбер; надо писать свою функцию для каждой сети
def preprocessing(system):
    return Data(**system)

In [8]:
#config
batch_size = 4
num_workers = 0

features_cols = ['feature_1']

target_col = 'y_relaxed'
lr = 0.001
epochs = 40

In [9]:
# #чтобы тензор по умолчанию заводился на куде
# if torch.cuda.is_available():
#     torch.set_default_tensor_type('torch.cuda.FloatTensor')
#     print('cuda')

In [10]:
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
print(device)

cuda


In [11]:
#инициализируем тренировочный датасети и тренировочный итератор
train_dataset_file_path = os.path.expanduser("../../ocp_datasets/data/is2re/10k/train/data_mod1.lmdbz")

training_set = Dataset(train_dataset_file_path, features_cols, target_col, preprocessing=preprocessing)
training_generator = DataListLoader(training_set, batch_size=batch_size)

In [12]:
#инициализируем валидационный датасет и валидационный итератор
val_dataset_file_path = os.path.expanduser("../../ocp_datasets/data/is2re/all/val_id/data_mod1.lmdbz")

valid_set = Dataset(val_dataset_file_path, features_cols, target_col, preprocessing=preprocessing)
valid_generator = DataListLoader(valid_set, batch_size=batch_size, num_workers=num_workers)

In [ ]:
try:
    lmdb_dataset(train_dataset_file_path).describe()
except:
    pass

In [13]:
#model
# model = spinconv(None, None, 1, otf_graph=False, regress_forces=False, use_pbc=False)
model = torch.load('checkpoints/2021-10-03-20-12-13_SpinConv_SolidAngles.pt')
model = DataParallel(model)

#optimizer and loss
optimizer = optim.AdamW(model.parameters(), lr=lr)
criterion = nn.L1Loss()

#переносим на куду если она есть
model = model.to(device)
criterion = criterion.to(device)

In [14]:
timestamp = str(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))

print(timestamp)

2021-10-06-06-53-54


In [15]:
#tensorboard writer, при первом запуске надо руками сделать папку для логов

# server
#log_folder_path = "../../ocp_results/logs/tensorboard/out_base_model"

# colab
# log_folder_path = "/content/drive/MyDrive/ocp_results/logs/tensorboard/out_base_model"

# user_specific 
log_file_path = "../logs/tensorboard_airi"

writer = SummaryWriter(log_file_path + '/' + timestamp)

In [16]:
%%time
logfile_str = {
    "train_dataset_file_path": train_dataset_file_path,
    "val_dataset_file_path": val_dataset_file_path,
    "features_cols": features_cols,
    "target_col": target_col,
    "batch_size": batch_size,
    "num_workers": num_workers,
    "epochs": epochs,
    "lr": lr,
    "type": "Solid angles"
}

#граф модели
try:
    #trace_system = dict(list(next(iter(training_generator))[0]))
    writer.add_graph(model, trace_system)
except:
    print('no graph')
writer.add_text(timestamp, str(logfile_str))

no graph
CPU times: user 1.63 ms, sys: 458 µs, total: 2.09 ms
Wall time: 1.79 ms


## Training

In [ ]:
%%time
loss = []
loss_eval = []

print(timestamp)
print(f'Start training model {str(model)}')
for i in range(epochs):
    loss.append(train(model, training_generator, optimizer, criterion, epoch=i, writer=writer, device=device))
    loss_eval.append(evaluate(model, valid_generator, criterion, epoch=i, writer=writer, device=device))

In [17]:
evaluate(model, training_generator, criterion, epoch=0, writer=writer, device=device)

epoch 0 evaluation


AttributeError: Caught AttributeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/share/miniconda3/envs/ocp_models/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/share/miniconda3/envs/ocp_models/lib/python3.8/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/share/miniconda3/envs/ocp_models/lib/python3.8/site-packages/torch_geometric/nn/data_parallel.py", line 67, in forward
    inputs = self.scatter(data_list, self.device_ids)
  File "/share/miniconda3/envs/ocp_models/lib/python3.8/site-packages/torch_geometric/nn/data_parallel.py", line 75, in scatter
    count = torch.tensor([data.num_nodes for data in data_list])
  File "/share/miniconda3/envs/ocp_models/lib/python3.8/site-packages/torch_geometric/nn/data_parallel.py", line 75, in <listcomp>
    count = torch.tensor([data.num_nodes for data in 